## Merge runs

In [2]:
print('Start import')
import socket 
import strax
import straxen

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import datetime
import pandas as pd
import time
import itertools
import os,sys
import cutax
import warnings
warnings.filterwarnings('ignore')
import copy

import local_minimum_info as lm
from local_minimum_info import *

straxen.print_versions()

Start import
Working on dali010.rcc.local with the following versions and installation paths:
python	v3.8.12	(default, Oct 12 2021, 13:49:34) [GCC 7.5.0]
strax	v1.1.2	/opt/XENONnT/anaconda/envs/XENONnT_2021.12.2/lib/python3.8/site-packages/strax
straxen	v1.1.3	/opt/XENONnT/anaconda/envs/XENONnT_2021.12.2/lib/python3.8/site-packages/straxen
cutax	v1.2.0	/dali/lgrandi/xenonnt/software/cutax/v1.2.0/cutax


In [3]:
st = cutax.contexts.xenonnt_v5()

In [4]:
dsets = st.select_runs(include_tags='_sr0_preliminary', run_mode='tpc_kr83m',available='event_info')
dsets = dsets[(dsets.end > pd.to_datetime('06/03/2021'))]

Fetching run info from MongoDB:   0%|          | 0/31547 [00:00<?, ?it/s]

Checking data availability:   0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
runs = np.array(dsets['name'])
print(len(runs))
print(runs)

418
['033301' '033298' '033295' '033292' '033289' '033286' '033283' '033280'
 '033277' '033274' '033271' '033268' '033265' '033262' '033259' '033256'
 '033253' '033250' '033247' '033244' '033241' '033238' '033235' '033232'
 '033229' '033226' '033223' '033220' '033217' '033214' '033211' '033208'
 '033205' '033202' '033199' '031963' '031960' '031957' '031954' '031951'
 '031948' '031945' '031942' '031939' '031936' '031933' '031930' '031927'
 '031924' '031921' '031918' '031915' '031912' '031909' '031906' '031903'
 '031900' '031897' '031894' '031891' '031888' '031885' '031882' '031879'
 '031876' '031873' '031870' '031867' '031864' '031861' '031858' '031855'
 '031852' '031849' '031846' '031843' '031840' '031837' '031834' '031317'
 '031314' '031311' '031308' '031305' '031302' '031299' '031296' '031293'
 '031290' '031287' '031284' '031281' '031278' '031275' '031272' '031269'
 '031266' '031263' '031260' '030565' '030562' '030559' '030556' '030553'
 '030550' '030547' '030544' '030541' '030538' '

In [7]:
path = '/dali/lgrandi/paetschb/data_backup/S2_studies/'
# df_new = pd.read_pickle(f'{path}SR0_v5_Kr83m_024418_031963_datasets.pkl')
# df_new = pd.read_pickle(f'{path}SR0_v5_Kr83m_026078_027171_EventInfoNew.pkl')
# df_new = pd.read_pickle(f'{path}SR0_v5_AmBe_021286_021461_EventInfoNew.pkl')
name='Kr83m'

In [5]:
def merge_runs_local_minimum_info_v5(st,runs,wire_cut=(4.45)):
    ''' In addition to all the cuts from the merge_runs_local_minimum_info we now also load the S2 widht cut!!
        The perp_wire_cut of 4.45 is the default value at the moment.
        Personally, I think 1.5 makes way more sense!! (11.5-14.5) cm
    '''
    event_bas0 = st.get_array(runs[0],['event_basics']) # we need them for local minimum info (?)
    st.register(LocalMinimumInfo)
    st.register(EventInfoNew)
    st.set_config(dict(perp_wires_cut_distance = wire_cut)) # xcut = (11.5, 14.5) cm
    ev0 = st.get_df(runs[0],['event_info_new',
                             'cut_s1_max_pmt',
                             'cut_s1_area_fraction_top',
                             'cut_s2_single_scatter',
                             'cut_s2_width',
                             'cut_fiducial_volume',
                             'cut_daq_veto'],progress_bar=False)
    print('Reading runs from',runs[-1],'to',runs[0])
    start = time.time()
    for i, run_id in enumerate(runs[1:]):
#         if ((i+1)%5) == 0: 
        print(f'n. {i} run {run_id} elapsed time: {time.time()-start:.2f} s')
        event_bas_temp = st.get_array(run_id,['event_basics']) # we need them for local minimum info (?)
        ev_temp = st.get_df(run_id,['event_info_new',
                             'cut_s1_max_pmt',
                             'cut_s1_area_fraction_top',
                             'cut_s2_single_scatter',
                             'cut_s2_width',
                             'cut_fiducial_volume',
                             'cut_daq_veto'],progress_bar=False)
        frames = [ev0,ev_temp]
        ev0 = pd.concat(frames)
    return ev0

In [ ]:
# creating merge runs
runs_small = runs[:220]
print(runs_small[0],runs_small[-1])
ev = merge_runs_local_minimum_info_v5(st,runs_small)
ev.to_pickle(f'{path}SR0_v5_Kr83m_{runs_small[-1]}_{runs_small[0]}_EventInfoNew_S2Width4p45.pkl')
# ev.to_pickle(f'{path}SR0_v6_Kr83m_{runs_small[-1]}_{runs_small[0]}_EventInfoNew_1p5.pkl')